In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import join
folder = join('..', 'data', 'laws')

In [3]:
import fastText
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
import sklearn
import pymorphy2
import tqdm
import random
from sklearn.model_selection import train_test_split

In [4]:
texts = []
for i in range(1,26):
    texts += open(join(folder, '{}.txt'.format(i)), encoding='utf-8')

In [5]:
s = "".join(texts)
s = re.sub(r"[^A-Za-zА-Яа-я]+", " ", s)

In [6]:
words = [word.lower() for word in s.split(" ")]

In [7]:
morph = pymorphy2.MorphAnalyzer()

In [8]:
# долго работает

words_ = []
for word in tqdm.tqdm_notebook(words):
    words_ += [morph.parse(word)[0].normal_form]

HBox(children=(IntProgress(value=0, max=1162270), HTML(value='')))

In [9]:
s = " ".join(words_)

In [10]:
tmp = open('tmp.txt', mode='w')
tmp.write(s)

9225499

In [11]:
words_ = np.unique(words_)
words_ = words_[1:]

In [12]:
model = fastText.train_unsupervised('tmp.txt')

In [13]:
X = [model.get_word_vector(word) for word in words_]

In [14]:
word2X = dict(zip(words_, X))

In [15]:
kNN = NearestNeighbors(n_neighbors=20).fit(X)

In [16]:
# нахождение 'синонимов' той же части речи

def find_syn(word):
    w = morph.parse(word)[0].normal_form
    try:
        ind = kNN.kneighbors([word2X[w]])[1]
    except KeyError:
        print("Данного слова нет в словаре")
        return
    for w_ in words_[ind].ravel():
        if morph.parse(w)[0].tag.cyr_repr.split(',')[0] == morph.parse(w_)[0].tag.cyr_repr.split(',')[0]:
            print(w_)

In [17]:
find_syn('утверждать')

утверждать
утверждаться
утвердить
дорабатывать
устанавливаться
выселять


**Классификация глаголов**

In [58]:
verbs = []

for word in words_:
    if morph.parse(word)[0].tag.cyr_repr.split(',')[0] == "ИНФ":
        verbs.append(word)
        print(word)

абортировать
адаптировать
адресовать
аккредитовать
аккумулировать
амортизировать
амортизироваться
анализировать
аннулировать
аннулироваться
арендовать
аттестовать
аффилировать
бальзамировать
бидти
блокировать
браковать
бросить
буксировать
быть
варить
ввезти
ввести
вводить
вводиться
ввозить
ведать
вернуть
вернуться
вести
вестись
взаимодействовать
взвесить
взимать
взиматься
взыскать
взыскивать
взыскиваться
взять
видеть
вкладывать
включать
включаться
включить
владеть
влечь
влиять
вложить
вменить
вменяться
вмешиваться
вмещать
внедрять
внести
вносить
вноситься
вовлекать
возбудить
возбуждать
возвести
возводить
возвратить
возвращать
возвращаться
возглавлять
возглавляться
возлагать
возлагаться
возложить
возместить
возмещать
возмещаться
возникать
возникнуть
возобновить
возобновлять
возобновляться
возражать
войти
воспитывать
воспользоваться
воспрепятствовать
воспрещаться
воспринимать
восстанавливать
восстанавливаться
восстановить
востребовать
вручать
вручаться
вручить
вскрыть
встать
встречать
вс

пользоваться
поместить
помещать
помещаться
понести
понижать
понизить
понимать
пониматься
попадать
пополняться
порождать
поручать
поручаться
поручить
поселкообразовать
поселкообразуть
посещать
последовать
послужить
поставить
поставлять
поставляться
постановлять
построить
поступать
поступить
посчитать
потерять
потребить
потреблять
потребляться
потребовать
правоудостоверять
правоустанавливать
пребывать
превосходить
превысить
превышать
предлагать
предлагаться
предложить
предназначаться
предназначить
предоставить
предоставлять
предоставляться
предотвращать
предписать
предполагать
предполагаться
предпринять
представить
представлять
представляться
предупредить
предупреждать
предупреждаться
предусматривать
предусматриваться
предусмотреть
предшествовать
предъявить
предъявлять
предъявляться
прекратить
прекращать
прекращаться
преобладать
преобразовать
преподавать
преподаваться
препятствовать
прерывать
прерываться
пресекать
претендовать
прибавлять
прибывать
прибыть
привести
привлекать
привлекаться

In [76]:
dict_words = dict(zip(verbs, np.zeros(len(verbs), dtype=int)))

In [77]:
with open('verbs.txt', 'r') as f:
    for verb in f.readlines():
        dict_words[verb[:-1]] = 1

In [78]:
dict_words

{'предписать': 0,
 'загрязнить': 0,
 'кипеть': 0,
 'районировать': 0,
 'способствовать': 1,
 'преподаваться': 0,
 'отобрать': 0,
 'передать': 1,
 'ссылаться': 0,
 'отгружаться': 0,
 'монтировать': 0,
 'отклоняться': 0,
 'использоваться': 0,
 'продаваться': 0,
 'заинтересовать': 0,
 'включать': 0,
 'предложить': 0,
 'понести': 0,
 'скупить': 0,
 'доказывать': 0,
 'подтвердить': 0,
 'длиться': 0,
 'предоставляться': 0,
 'следовать': 0,
 'маркировать': 0,
 'обжаловать': 1,
 'присоединиться': 0,
 'удовлетворять': 0,
 'получаться': 0,
 'оснастить': 0,
 'пребывать': 0,
 'совместить': 0,
 'оспаривать': 0,
 'проследовать': 0,
 'выражать': 1,
 'дополняться': 0,
 'полагаться': 0,
 'оглашаться': 0,
 'скрыться': 0,
 'отражать': 0,
 'аннулировать': 1,
 'отбывать': 0,
 'допрашиваться': 0,
 'соединить': 0,
 'огородить': 0,
 'отразить': 0,
 'пропагандировать': 0,
 'характеризоваться': 0,
 'преобразовать': 0,
 'сопрячь': 0,
 'согласовываться': 0,
 'избираться': 0,
 'образовать': 0,
 'расходовать': 0,
 

In [79]:
X = [model.get_word_vector(word) for word in verbs]

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, list(dict_words.values()), test_size=0.3)

In [81]:
from sklearn.linear_model import LogisticRegression


In [82]:
lr = LogisticRegression(C=1000)
lr.fit(X_train, y_train)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [83]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([735, 147]))

In [84]:
y_pred = lr.predict(X_test)

In [85]:
from sklearn.metrics import *

In [86]:
accuracy_score(y_test, y_pred)

0.8095238095238095